In [25]:
import pandas as pd
import numpy as np
import jieba 

from keras.layers import Dense, Input, Flatten,Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model

In [2]:
fneg = open('neg.txt','r',encoding='gb18030')
fpos = open('pos.txt','r',encoding='gb18030')

In [3]:
neg = []
pos = []
for line in fneg:
    neg.append(line)
for line in fpos:
    pos.append(line)

In [4]:
neg_df = pd.DataFrame(neg)
pos_df = pd.DataFrame(pos)

In [5]:
#合并语料

pn = pd.concat([neg_df, pos_df], ignore_index=True)
neg_len = len(neg_df)
pos_len = len(pos_df)

In [6]:
#定义分词函数

cw = lambda x: list(jieba.cut(x))
pn['words'] = pn[0].apply(cw)

Building prefix dict from the default dictionary ...
I0728 17:47:16.336415 17004 __init__.py:111] Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Andy\AppData\Local\Temp\jieba.cache
I0728 17:47:16.342421 17004 __init__.py:131] Loading model from cache C:\Users\Andy\AppData\Local\Temp\jieba.cache
Loading model cost 2.955 seconds.
I0728 17:47:19.296510 17004 __init__.py:163] Loading model cost 2.955 seconds.
Prefix dict has been built succesfully.
I0728 17:47:19.302014 17004 __init__.py:164] Prefix dict has been built succesfully.


In [7]:
#将一行最大单词数设为1000
max_document_length = 1000

In [8]:
texts = [' '.join(x) for x in pn['words']]

In [13]:
#实例化分词器
tokenizer = Tokenizer(num_words=30000)
#建立词典
tokenizer.fit_on_texts(texts)
#将词频和序号对应
sequences = tokenizer.texts_to_sequences(texts)
#将序列设置为1000的长度，超过1000的部分舍弃，不足的补0 补0的位置和CNN不同
sequences = pad_sequences(sequences, maxlen=1000)
#sequences = np.array(sequences)
type(sequences)

numpy.ndarray

In [14]:
#词对应编号的词典

dict_text = tokenizer.word_index
dict_text['天']

346

In [15]:
#这里的x是句子中每个分词对应的word_index的列表

#定义标签
positive_labels = [[0, 1] for _ in range(pos_len)]
negative_labels = [[1, 0] for _ in range(neg_len)]
y = np.concatenate([positive_labels, negative_labels], 0)
#打乱数据
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = sequences[shuffle_indices]
y_shuffled = y[shuffle_indices]

#数据集切分
test_sample_index = -1 * int(0.15 * float(len(y)))
x_train, x_test = x_shuffled[:test_sample_index], x_shuffled[test_sample_index:]
y_train, y_test = y_shuffled[:test_sample_index], y_shuffled[test_sample_index:]

In [17]:
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers.recurrent import GRU

In [32]:
#模型输入
sequence_input = Input(shape=(1000,))
#Embedding层，30000表示30000个词，每个词对应向量128维，序列长度1000
embedding_layer = Embedding(
    30000,  #生成一个30000*128的矩阵
    128, 
    input_length=1000
)
embedding_sequences = embedding_layer(sequence_input)

#方法一 LSTM输出为(batch,10)
lstm1 = LSTM(10, dropout=0.2, recurrent_dropout=0.2)(embedding_sequences) #10代表10个block
lstm1 = Dense(16, activation='relu')(lstm1)
lstm1 = Dropout(0.5)(lstm1)

#方法二 LSTM输出为（batch,1000,10)
lstm2 = LSTM(10,return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(embedding_sequences)
lstm2 = Flatten()(lstm2)
lstm2 = Dense(16, activation='relu')(lstm2)
lstm2 = Dropout(0.5)(lstm2)



##输出层

preds = Dense(2, activation='softmax')(lstm2)
model = Model(sequence_input, preds)

In [33]:
#编译和训练
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy',
    #loss = 'mse',
    metrics=['accuracy'])

In [34]:
#训练,60000张图片训练一次叫做一个epoch
model.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
8500/8500 [==============================] - 442s 52ms/step - loss: 0.4505 - acc: 0.7735
Epoch 2/10
8500/8500 [==============================] - 386s 45ms/step - loss: 0.2003 - acc: 0.9314
Epoch 3/10
8500/8500 [==============================] - 350s 41ms/step - loss: 0.1072 - acc: 0.9679
Epoch 4/10
8500/8500 [==============================] - 396s 47ms/step - loss: 0.0736 - acc: 0.9788
Epoch 5/10
8500/8500 [==============================] - 393s 46ms/step - loss: 0.0516 - acc: 0.9826
Epoch 6/10
8500/8500 [==============================] - 398s 47ms/step - loss: 0.0339 - acc: 0.9887
Epoch 7/10
8500/8500 [==============================] - 431s 51ms/step - loss: 0.0421 - acc: 0.9880
Epoch 8/10
8500/8500 [==============================] - 455s 54ms/step - loss: 0.0287 - acc: 0.9902
Epoch 9/10
8500/8500 [==============================] - 447s 53ms/step - loss: 0.0168 - acc: 0.9955
Epoch 10/10
8500/8500 [==============================] - 373s 44ms/step - loss: 0.0160 - acc: 0.9948

In [31]:
loss, accuracy = model.evaluate(x_train, y_train)
print("Loss:", loss)
print("Accuracy:", accuracy)

8500/8500 [==============================] - 60s 7ms/step
Loss: 0.010887666234143955
Accuracy: 0.9970588235294118
